# Sparklehorse - Der SQL-Chatbot der Magpie

## 1. Einleitung

Hintergrund (Todo)

Als Datenbank wird eine mview genutzt, die auf der Magpie basiert. Diese mview soll regelmäßig aktualisiert werden, um die neuesten Daten zu reflektieren. Der Chatbot ist so konzipiert, dass er SQL-Abfragen generieren kann, die auf den in der mview gespeicherten Daten basieren. 

## 2. Vorbereitung Sparklehorse

### 2.1 Arbeitsverzeichnis

In einem ersten Schritt definieren wir unser Arbeitsverzeichnis. 

In [ ]:
import os
os.getcwd()
os.chdir("c:/Users/mhu/Documents/gitHub/magpie_chatbot")

### 2.2 API Key

Wir laden unsere Umgebungsvariablen (inkl. OpenAI-API-Key) und initialisiere den Chatbot mit dem Modell "gpt-4o" von OpenAI.

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
llm = ChatOpenAI(model="gpt-4o")

### 2.3 Verbindung zur Magpie

Im folgenden stellen wir Verbindung zur Magpie her.

In [ ]:
import pandas as pd
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit 

db = SQLDatabase.from_uri("duckdb:///data/view_magpie.db")

## 3. Sparklehorse Tools

Im LangChain-Framework sind Tools Funktionen oder Schnittstellen, die ein Agent (hier Sparklehorse) aufrufen kann, um bestimmte Aufgaben außerhalb des reinen Textverstehens zu erledigen. Sie erweitern die Fähigkeiten des Agenten flexibel. 

### 3.1 Standardisierte Langchain Tools

Wir initialisieren ein standardisiertes Toolkit. Es stellt Tools bereit, um SQL-Queries über natürliche Sprache zu erzeugen und auszuführen. Wir lassen uns Namen und Funktion der standardisierten Tools anzeigen:

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"Tool Name: {tool.name}")
    print(f"Description: {tool.description}")
    print("-" * 40)

Die Standardtools aus`SQLDatabaseToolkit` können also die folgenden Standardfunktionen übernehmen: 

- `sql_db_query`  
  Führt eine übergebene SQL-Abfrage aus. Gibt das Ergebnis oder eine Fehlermeldung zurück. Bei Fehlern wie „Unknown column“ sollte zuvor das Tabellenschema geprüft werden.

- `sql_db_schema`  
  Gibt das Schema (Spaltennamen und -typen) sowie Beispielzeilen für angegebene Tabellen zurück. Vorher sollte geprüft werden, ob die Tabellen existieren.

- `sql_db_list_tables`  
  Listet alle Tabellen in der verbundenen Datenbank auf.

- `sql_db_query_checker`  
  Prüft eine SQL-Abfrage auf syntaktische Korrektheit, bevor sie mit sql_db_query ausgeführt wird. Sollte immer vorher verwendet werden.


### 3.2 Maßgeschneiderte Langchain Tools

#### 3.2.1 Tool Nr.1: `variable_beschr`

Das Tool `variable_beschr` soll es ermöglichen, aus der eingegebenen Frage eines Nutzers die korrekte Variable aus der Magpie zu identifizieren. Dazu verwendet `variable_beschr` (1) den `rt_beschr_variable`-Retriever: `rt_beschr_variable` erlaubt die semantischen Suche über Werte aus der Variable `variable_beschr`: `rt_beschr_variable`  sammelt sämtliche unique Werte aus `beschr_variable` und wandelt diese mit OpenAIs Embeddings-Methode `text-embedding-3-large` in Embeddings um. Diese werden in einen Vektorstore gesichert. Der Vektorstore wird in einen Retriever umgewandelt, der bei einer Anfrage die `n=10` ähnlichsten Begriffe zurückgibt. Schließlich wird mit `create_retriever_tool` ein Tool erzeugt, das den Retriever kapselt. Dieses Tool wird von `variable_beschr` genutzt, um Benutzereingaben mit unsicherer Schreibweise oder unvollständigen Begriffen mit den tatsächlichen Werten der Variablen in der Magpie abzugleichen.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.agents.agent_toolkits import create_retriever_tool
import ast
import re

def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [string.strip() for string in res]
    return list(set(res))


beschr_variable = query_as_list(db, "SELECT variable_beschr FROM view_daten_reichweite_menge")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_texts(beschr_variable)

retriever_beschr_variable  = vector_store.as_retriever(search_kwargs={"k": 10})

description = (
    "Verwenden, um Werte für Filterabfragen nachzuschlagen. Die Eingabe ist eine ungefähre Schreibweise "
    "eines Eigennamens, die Ausgabe sind gültige Eigennamen. Verwende den Begriff, der der Eingabe am ähnlichsten ist."
)

rt_beschr_variable = create_retriever_tool(
    retriever_beschr_variable,
    name="rt_beschr_variable",
    description=description,
)

#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
# Comment: Test des Retriever-Tools
#:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

def print_clean_result(result):
    print("\n".join(result.split("\n\n")))

result = rt_beschr_variable.invoke("Wie hoch waren die externe fue-aufwendungen, im Jahr 2020, in Deutschland, im wirtschaftssektor, bei forschungsintensive wirtschaftszweige Forschung?")

print_clean_result(result)

Das eigentliche Tool `variable_beschr` geht wie folgt vor: Es nimmt eine Nutzerfrage entgegen und verwendet den `rt_beschr_variable`-Retriever, um mithilfe von Embeddings eine Liste relevanter Variablenbeschreibungen aus einer vektorisierten Dokumentensammlung zu finden. Falls keine passenden Kandidaten gefunden werden, gibt das Tool eine Fehlermeldung zurück. Andernfalls wird ein Prompt generiert, der das Sprachmodell auffordert, exakt eine Variable aus dieser Liste auszuwählen – jedoch nur, wenn diese wirklich präzise zur eingegebenen Frage passt. Das Modell gibt daraufhin den exakten Text der ausgewählten Variable zurück. Dieser wird anschließend in einer SQL-Abfrage verwendet, um zu überprüfen, ob die Variable in der Datenbank vorhanden ist. Wird sie gefunden, gibt das Tool sie zurück. Falls nicht, wird eine Rückmeldung generiert, die den Nutzer zur genaueren Spezifikation auffordert.

Warum genügt nicht der einfache Retriever `rt_beschr_variable` für das Suchen der korrekten Variable? Im Rahmen des Testings hat sich gezeigt, dass die Korrekte Variable nicht umnbedingt jene ist, die die stärkste semantische Ähnlichkeit zur Nutzerfrage aufweist. Daher wird das Tool `variable_beschr` benötigt, um die Variable zu identifizieren, die auch inhatlich am besten zur Frage passt.

In [ ]:
from langchain_core.tools import tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

@tool
def variable_beschr(user_question: str) -> str:
    """
    Nutzt ein LLM und Embeddings, um aus der Frage eine passende Variable zu bestimmen
    und gibt dann die exakte Variable aus der Datenbank zurück.
    """
    docs = retriever_beschr_variable.get_relevant_documents(user_question)
    if not docs:
        return "Error: Keine passende Variable gefunden."

    kandidaten = "\n".join(f"- {doc.page_content.strip()}" for doc in docs)
    print(kandidaten)

    auswahl_prompt = PromptTemplate(
        input_variables=["frage", "kandidaten"],
        template="""
    Wähle exakt **eine** der folgenden Variablen, die am besten zur Frage passt.
    Wähle **nur dann** eine Variable aus, wenn sie **exakt** zur Frage passt.
    Nutze **keine verwandten Begriffe**, Oberkategorien oder Synonyme.
    Gib den Text **genau so** zurück, wie er bei den Kandidaten steht.

    Frage: {frage}

    Kandidaten:
    {kandidaten}

    Beste Variable:
    """
    )
    auswahl_chain = auswahl_prompt | llm
    best_match = auswahl_chain.invoke({
        "frage": user_question,
        "kandidaten": kandidaten
    }).content.strip()

    query = f"""
        SELECT variable_beschr 
        FROM view_daten_reichweite_menge 
        WHERE variable_beschr = '{best_match}' 
        LIMIT 1;
    """
    result = db.run_no_throw(query)
    if result:
        return result
    else:
        return "[USER_CLARIFICATION_NEEDED] Ich konnte keine passende Variable finden. Bitte geben Sie die gewünschte Variable genauer an."

Wir testen nun das Tool Nr.1:

In [ ]:
test_input = "Wie hoch waren die externe fue-aufwendungen, im Jahr 2020, in Deutschland, im wirtschaftssektor, bei forschungsintensive wirtschaftszweige Forschung?"
output = variable_beschr(test_input)
print(output)

#### 3.2.2 Tool Nr.2: `rt_reichweite_variable`

  Wir bauen einen ähnlichen Retriever nun auch für die Variablen `reichweite_beschr_list`. Das Tool `get_reichweite_beschr_list` bestimmt aus der Nutzerfrage die passende Reichweite zu einer Variable. Dazu nutzt es zuerst `variable_beschr`, das aus der Frage die exakte Variable ermittelt (siehe oben). Anschließend fragt es mit dieser Variable die Datenbank nach möglichen Reichweiten ab. Diese Reichweiten werden dann mit einem Vektor-Speicher und Text-Embeddings semantisch mit der Nutzerfrage verglichen, um die fünf relevantesten Kandidaten zu finden. Mithilfe eines Few-Shot-Prompts, das dem Modell anhand von Beispielen zeigt, wann `Deutschland` trotz semantisch ähnlicher anderer Reichweiten die richtige Wahl ist, wählt das Sprachmodell die beste Reichweite aus. Wird keine passende Reichweite gefunden oder ist die Auswahl ungültig, fordert das Tool eine genauere Eingabe vom Nutzer an. Abschließend wird die gewählte Reichweite noch einmal über die Datenbank validiert und als Ergebnis zurückgegeben. So kombiniert das Tool Datenbankabfragen, semantische Suche und gezielte Steuerung des Modells, um die passende Reichweite kontextsensitiv zu ermitteln.


In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

reichweiten_beispiele = [
    {"frage": "Wie viele Absolventen für Berufliche Schulen gab es?", "variable_beschr": "Anzahl der Absolventen für Berufliche Schulen", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie hoch war die Studierquote bildungsferner Schichten?", "variable_beschr": "Studierquote bildungsferne Schichten", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie viele dauerhaft eingestellte Lehrkräfte (inkl. Seiteneinsteigern, ohne Referendare) gab es?", "variable_beschr": "Anzahl dauerhaft eingestellte Lehrkräfte (inkl. Seiteneinsteigern, ohne Referendare)", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie hoch war der Handlungsfeldindex: Lehrer Bildung?", "variable_beschr": "Handlungsfeldindex: Lehrer Bildung", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie viele Universitätsschulverbünde gab es?", "variable_beschr": "Anzahl Universitätsschulverbünde", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie hoch war der Anteil berufsbegleitender Master?", "variable_beschr": "Anteil berufsbegleitender Master", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie viele Studienabsolventen T gab es?", "variable_beschr": "Studienabsolventen T", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie hoch waren die internen FuE-Aufwendungen?", "variable_beschr": "Interne FuE-Aufwendungen", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie hoch war der Anteil der männlichen Grundschullehramtsstudierenden?", "variable_beschr": "Anteil der männlichen Grundschullehramtsstudierende", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie viele Studienabsolventen im Weiterbildungsstudium gab es?", "variable_beschr": "Studienabsolventen im Weiterbildungsstudium", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie hoch waren die Drittmittel vom Bund 2021 in Deutschland?", "variable_beschr": "Drittmittel vom Bund", "reichweite_beschr_list": "Deutschland"}
]

example_prompt = PromptTemplate(
    input_variables=["frage", "variable_beschr", "reichweite_beschr_list"],
    template="Frage: {frage}\nVariable: {variable_beschr}\n→ Reichweite: {reichweite_beschr_list}"
)

reichweite_prompt = FewShotPromptTemplate(
    examples=reichweiten_beispiele,
    example_prompt=example_prompt,
    prefix="Wähle aus den möglichen Reichweiten die beste. Nutze 'Deutschland', wenn keine Region, Organisation o. Ä. genannt wird.",
    suffix="Frage: {frage}\nVariable: {variable_beschr}\nKandidaten:\n{kandidaten}\n→ Reichweite:",
    input_variables=["frage", "variable_beschr", "kandidaten"]
)

In [ ]:
@tool
def get_reichweite_beschr_list(user_question: str) -> str:
    """
    Ermittelt eine passende Reichweite (z. B. Region, Organisation, etc.), basierend auf der
    zur Frage gehörigen Variable und den verfügbaren Einträgen in der Datenbank.
    """
    print("[DEBUG] Eingabe-Frage:", user_question)

    raw_variable = variable_beschr.run(user_question)
    print("[DEBUG] raw_variable:", raw_variable)

    match = re.search(r"'([^']+)'", str(raw_variable))
    if not match:
        print("[DEBUG] Abbruch: Keine gültige Variable extrahiert")
        return "Fehler: Konnte keine gültige Variable bestimmen."

    variable = match.group(1)
    print("[DEBUG] bereinigte variable:", variable)

    if "Error" in variable:
        return "Fehler: Konnte keine gültige Variable bestimmen."

    escaped_variable = variable.replace("'", "''")
    print("[DEBUG] escaped_variable:", escaped_variable)

    query = f"""
        SELECT DISTINCT reichweite_beschr_list 
        FROM view_daten_reichweite_menge 
        WHERE variable_beschr = '{escaped_variable}'
    """
    print("[DEBUG] SQL-Abfrage gültige_reichweiten:", query)
    gültige_reichweiten = query_as_list(db, query)
    print("[DEBUG] gültige_reichweiten:", gültige_reichweiten)

    if not gültige_reichweiten:
        return "[USER_CLARIFICATION_NEEDED] Ich konnte keine passende Reichweite ermitteln. Bitte präzisieren Sie, welche Region oder Organisation gemeint ist."


    vector_store = InMemoryVectorStore(OpenAIEmbeddings(model="text-embedding-3-large"))
    _ = vector_store.add_texts(gültige_reichweiten)
    retriever = vector_store.as_retriever(search_kwargs={"k": 5})

    top_matches = retriever.get_relevant_documents(user_question)
    reichweiten_kandidaten = [doc.page_content for doc in top_matches]
    print("[DEBUG] Top 5 Reichweiten-Kandidaten:", reichweiten_kandidaten)

    kandidaten_text = "\n".join(reichweiten_kandidaten)

    llm_chain = reichweite_prompt | llm
    best_match = llm_chain.invoke({
        "frage": user_question,
        "variable_beschr": variable,
        "kandidaten": kandidaten_text
    }).content.strip()

    print("[DEBUG] LLM-best_match:", best_match)

    # Validierung: nur erlaubte Rückgabe
    if best_match not in gültige_reichweiten:
        print(f"[DEBUG] LLM-Match ungültig ('{best_match}'), Rückfrage erforderlich")
        return "[USER_CLARIFICATION_NEEDED] Ich konnte keine passende Reichweite ermitteln. Bitte konkretisieren Sie Ihre Anfrage."
        
    query = f"""
        SELECT reichweite_beschr_list 
        FROM view_daten_reichweite_menge 
        WHERE reichweite_beschr_list = '{best_match}' 
        LIMIT 1;
    """
    print("[DEBUG] SQL-Abfrage finale Auswahl:", query)
    result = db.run_no_throw(query)
    print("[DEBUG] Ergebnis:", result)

    return result if result else "Error: Keine passende Reichweite gefunden."

tools.extend([variable_beschr, get_reichweite_beschr_list])

Wir testen nun das Tool Nr.1:

In [ ]:
test_input = "Wie hoch waren die externe fue-aufwendungen, im Jahr 2020, in Deutschland, im wirtschaftssektor, bei forschungsintensive wirtschaftszweige Forschung?"
output = get_reichweite_beschr_list(test_input)
print(output)

 ## 4. Sparklehorse System-Prompt-Template

Im folgenden wird ein Prompt-Template aus dem LangChain Hub geladen und an den spezifischen Anwendungsfall angepasst. Zunächst wird das Standard-Systemprompt für einen SQL-Agenten mit `hub.pull("langchain-ai/sql-agent-system-prompt")` importiert. Dieses Prompt besteht aus genau einer Nachricht, was durch eine `assert`-Anweisung sichergestellt wird.

Anschließend wird diese Nachricht um eine spezifische Beschreibung erweitert: Das Sprachmodell wird als "Sparklehorse" definiert – ein Chatbot im Kontext der Organisation Stifterverband. Zusätzlich wird festgelegt, dass sich der Bot auf Fragen zur Magpie-Datenbank konzentrieren soll. Dieser Zusatz wird direkt an den bestehenden Prompt-Text angehängt.

Abschließend wird das modifizierte Prompt-Template mit `pretty_print()` ausgegeben, um den finalen Text zu überprüfen. Dadurch wird sichergestellt, dass das LLM im richtigen Kontext arbeitet und auf den spezifischen Anwendungsfall vorbereitet ist.

In [ ]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1, "Die Anzahl der Nachrichten im Template ist nicht 1!"
# Bearbeite die bestehende Nachricht, indem du Text hinzufügst
prompt_template.messages[0].prompt.template += (
    "\nYou are Sparklehorse, a chatbot for the Stifterverband organization. "
    "Your primary task is to answer questions related to the Magpie database."
)

system_message = prompt_template.format(
    dialect=db.dialect, 
    top_k=5
)

print(system_message)

## 5. Sparklehorse ReAct-Agenten  

In diesem Abschnitt wird ein spezialisierter ReAct-Agent erstellt, der strikt definierte Regeln für den Umgang mit SQL-Abfragen im Kontext der Magpie-Datenbank befolgen soll. Dazu wird zunächst ein ausführliches Regelwerk als `suffix` definiert. Dieses enthält genaue Anweisungen, wie der Agent mit den Tools `variable_beschr` und `get_reichweite_beschr_list` arbeiten soll, welche SQL-Syntax erlaubt ist, welche Felder genutzt werden dürfen und welche logischen Anforderungen an Konsistenz, Transparenz und Datenbezug gestellt werden.

Der `suffix` wird an die bestehende Systemnachricht (`system_message`) angehängt und ergibt zusammen den vollständigen Kontext, in dem das Sprachmodell operieren soll. Das sorgt dafür, dass der Agent sich vollständig an die definierten Rahmenbedingungen hält – etwa durch die Verwendung exakter Feldnamen, das Filtern nach Jahren mittels `date_part`, die Berücksichtigung von `wert_einheit` sowie die Pflicht, nur gültige Rückgaben der Tools in der SQL-Abfrage zu verwenden.

Abschließend wird mit `create_react_agent` ein neuer ReAct-Agent auf Basis des spezifizierten Sprachmodells (`llm`), der zur Verfügung stehenden Tools (`tools`) und des zusammengesetzten Systemkontexts (`system`) erzeugt. Dieser Agent ist damit gezielt darauf ausgerichtet, konsistente, nachvollziehbare und kontextgerechte Antworten für SQL-Abfragen zur Magpie-Datenbank zu liefern.

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

# Systemnachricht mit extra Anweisungen
suffix = (
    "1. Nutze das Tool `variable_beschr`, um die korrekte Variable aus der Nutzerfrage zu bestimmen. Verwende ausschließlich den exakten Rückgabewert dieses Tools für `variable_beschr` in der SQL-Abfrage.\n"
    "2. Nutze das Tool `get_reichweite_beschr_list`, um die passende Reichweite zu ermitteln. Verwende ausschließlich den Rückgabewert dieses Tools für `reichweite_beschr_list` in der SQL-Abfrage.\n"
    "3. Verwende **niemals** andere Felder wie `tag_list` oder `LIKE`-Abfragen. Nutze **immer exakte Vergleiche** mit `=`.\n"
    "4. Verwende ausschließlich die Tabelle `view_daten_reichweite_menge` für alle Abfragen.\n"
    "5. Falls ein Jahr in der Frage genannt wird, filtere mit `date_part('year', zeit_start) = <Jahr>`.\n"
    "6. Berücksichtige die Spalte `wert_einheit`, z. B. 'in Tsd. Euro', 'Anzahl', 'Prozent', 'VZÄ', 'Mitarbeiter'.\n"
    "7. Gib immer die finale SQL-Abfrage vollständig aus und erkläre sie. Rate niemals IDs oder Werte.\n"
    "8. Falls keine passende Variable oder Reichweite gefunden wurde, rate nicht irgendwelche Werte.\n"
    "9. Stelle sicher, dass Antworttext und SQL-Abfrage immer auf den gleichen `variable_beschr`- und `reichweite_beschr_list`-Werten basieren, um Konsistenz zu gewährleisten.\n"
    "10. Verwende in deiner Antwort exakt die Begriffe, die du in der SQL-Abfrage benutzt hast. Nutze insbesondere den Wert aus `reichweite_beschr_list` vollständig im Antwortsatz. Beispiel: Wenn `reichweite_beschr_list = 'Wirtschaftssektor | Deutschland'`, schreibe: 'im Wirtschaftssektor in Deutschland'."
)

system = f"{system_message}\n\n{suffix}"

# Neuen ReAct-Agent erstellen mit den vollständigen Tools
agent_executor = create_react_agent(llm, tools, state_modifier=system)


### 5.1 Streaming eines ReAct-Agenten mit Rückfrage-Erkennung

Die Funktion `stream_agent_with_check` dient dazu, eine Nutzerfrage an den zuvor definierten ReAct-Agenten zu übergeben und die Antwort schrittweise im Streaming-Modus auszugeben. Dabei wird das LLM mit der übergebenen Frage (`question`) in Form einer `HumanMessage` aufgerufen. Der Agent wird im `stream_mode="values"` betrieben, wodurch einzelne Antwortschritte direkt verarbeitet werden können, während sie generiert werden.

Innerhalb der Schleife wird jeweils die letzte generierte Nachricht (`step["messages"][-1]`) geprüft. Sollte diese die Markierung `[USER_CLARIFICATION_NEEDED]` enthalten, bedeutet dies, dass das Modell eine Rückfrage an den Nutzer stellen möchte – z. B. weil keine eindeutige Variable oder Reichweite gefund


In [ ]:
def stream_agent_with_check(question: str):
    stream = agent_executor.stream({"messages": [HumanMessage(content=question)]}, stream_mode="values")
    for step in stream:
        msg = step["messages"][-1]
        if "[USER_CLARIFICATION_NEEDED]" in msg.content:
            rückfrage = msg.content.replace("[USER_CLARIFICATION_NEEDED]", "").strip()
            print(f"⚠️ Rückfrage: {rückfrage}")
            break
        else:
            msg.pretty_print()

question = "Wie hoch waren die externe fue-aufwendungen, im Jahr 2020, in Deutschland, im wirtschaftssektor, bei forschungsintensive wirtschaftszweige Forschung?"

stream_agent_with_check(question)

## 6. Sparklehorse - Kontextbehaftete Unterhaltung mit Verlauf

`ask_with_memory` ermöglicht eine Unterhaltung mit dem Sparklehorse, bei der der Gesprächsverlauf erhalten bleibt:

- `chat_history` speichert alle bisherigen Nachrichten (Nutzer- und KI-Antworten).
- Die Funktion `ask_with_memory(user_input)`:
  - Fügt die neue Nutzereingabe als `HumanMessage` zum Verlauf hinzu.
  - Ruft den Agenten mit dem gesamten bisherigen Verlauf auf, damit das Modell den Kontext kennt.
  - Speichert die Antwort (`AIMessage`) des Agenten ebenfalls im Verlauf.
  - Gibt die Antwort als Text zurück.

So kann der Chatbot frühere Aussagen berücksichtigen und führt ein zusammenhängendes Gespräch.

In [17]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []

def ask_with_memory(user_input):
    chat_history.append(HumanMessage(content=user_input))
    response = agent_executor.invoke({"messages": chat_history})
    reply = response["messages"][-1]
    chat_history.append(reply)
    return reply.content

ask_with_memory(question)

[DEBUG] Eingabe-Frage: Wie hoch waren die externe fue-aufwendungen, im Jahr 2020, in Deutschland, im wirtschaftssektor, bei forschungsintensive wirtschaftszweige Forschung?
- Externe FuE-Aufwendungen
- FuE-Aufwendungen (vom Wirtschaftssektor finanziert)
- FuE-Aufwendungen (vom Staatssektor finanziert)
- Externe FuE für nicht verbundene Unternehmen (Inland)
- Externe FuE (Netto)
- Externe FuE für verbundene Unternehmen (Inland)
- Externe FuE für staatliche Forschungsinstitute (Inland)
- Interne FuE-Aufwendungen
- Externe FuE (Wirtschaft)
- Externe FuE für sonstige Institutionen und Unternehmen (Ausland)
- Externe FuE-Aufwendungen
- FuE-Aufwendungen (vom Wirtschaftssektor finanziert)
- FuE-Aufwendungen (vom Staatssektor finanziert)
- Externe FuE für nicht verbundene Unternehmen (Inland)
- Externe FuE (Netto)
- Externe FuE für verbundene Unternehmen (Inland)
- Externe FuE für staatliche Forschungsinstitute (Inland)
- Interne FuE-Aufwendungen
- Externe FuE (Wirtschaft)
- Externe FuE für so

'Im Jahr 2019 betrugen die externen FuE-Aufwendungen im Wirtschaftssektor in Deutschland bei forschungsintensiven Wirtschaftszweigen 19.253.898 Tausend Euro.'

In [18]:
ask_with_memory("Und im Jahr davor?")

[DEBUG] Eingabe-Frage: externe fue-aufwendungen im Jahr 2018 im wirtschaftssektor in Deutschland bei forschungsintensive wirtschaftszweige Forschung
- Externe FuE-Aufwendungen
- Externe FuE für nicht verbundene Unternehmen (Inland)
- FuE-Aufwendungen (vom Wirtschaftssektor finanziert)
- Externe FuE (Netto)
- Externe FuE für verbundene Unternehmen (Inland)
- Externe FuE (Wirtschaft)
- FuE-Aufwendungen (vom Staatssektor finanziert)
- Interne Aufwendungen für angewandte Forschung
- Externe FuE für sonstige Institutionen und Unternehmen (Ausland)
- Interne FuE-Aufwendungen
- Externe FuE-Aufwendungen
- Externe FuE für nicht verbundene Unternehmen (Inland)
- FuE-Aufwendungen (vom Wirtschaftssektor finanziert)
- Externe FuE (Netto)
- Externe FuE für verbundene Unternehmen (Inland)
- Externe FuE (Wirtschaft)
- FuE-Aufwendungen (vom Staatssektor finanziert)
- Interne Aufwendungen für angewandte Forschung
- Externe FuE für sonstige Institutionen und Unternehmen (Ausland)
- Interne FuE-Aufwendung

'Im Jahr 2018 betrugen die externen FuE-Aufwendungen im Wirtschaftssektor in Deutschland bei forschungsintensiven Wirtschaftszweigen 22.168.058 Tausend Euro.'

## 7. Test Sparklehorse

Um einen umfassenderen Leistungstest des Sparklehorse zu erzeugen, lesen wir einen Test-Datensatz ein, der Fragen zur Magpie beinhaltet und auch die Korrekten antworten enthält.

In [ ]:
import pandas as pd
from langchain.schema import HumanMessage

df = pd.read_excel("data/test_quest_mixed_nr_5.xlsx").sample(n=20, random_state=1)
 
for question in df["Frage"].dropna():
    result = agent_executor.invoke({"messages": [HumanMessage(content=question)]})
    messages = result["messages"]
    antwort = messages[-1].content
    print(f"\nFrage: {question}\nAntwort: {antwort}")
